<header style="width:100%;position:relative">
  <div style="width:80%;float:right;">
    <h1>Course Notes for Learning Intelligent Systems</h1>
    <h3>Department of Telematic Engineering Systems</h3>
    <h5>Universidad Politécnica de Madrid</h5>
  </div>
        <img style="width:15%;" src="../logo.jpg" alt="UPM" />
</header>

## Introduction to Linked Open Data

This lecture provides a quick introduction to semantic queries in Python using SPARQL.
SPARQL is aa semantic query language inspired by SQL.

This is the first in a series of notebooks about SPARQL, which consists of:

* This notebook, which introduces basic concepts using a small public dataset.
* [A notebook with queries to a custom dataset](02_SPARQL_Custom_Endpoint.ipynb), which links to the RDF exercises.
* [A notebook with queries to DBpedia](03_SPARQL_Writers.ipynb). DBpedia is the semantic version of Wikipedia. It is very useful, as it contains much more data. However, finding the right properties to query can be challenging.
* [A notebook with more advanced SPARQL concepts](04_SPARQL_Advanced.ipynb), which extends the previous notebook with more advanced concepts, such as regular expressions and dealing with dates.

## Objectives

* Learning SPARQL and the Linked Data principles by defining queries to answer a set of problems of increasing difficulty
* Learning how to use integrated SPARQL editors and programming interfaces to SPARQL.

## Tools

* This notebook
* External SPARQL editors (optional)
    * YASGUI-GSI http://yasgui.cluster.gsi.dit.upm.es
    * DBpedia virtuoso http://dbpedia.org/sparql

Using the YASGUI-GSI editor has several advantages over other options.
It features:

* Selection of data source, either by specifying the URL or by selecting from a dropdown menu
* Interactive query editing
    * A set of pre-defined queries
    * Syntax errors
    * Auto-complete
* Data visualization
    * Total number of results
    * Different formats (table, pivot table, raw response, etc.)
    * Pagination of results
    * Search and filter results

## Instructions

We will be using a semantic server, available at: http://fuseki.cluster.gsi.dit.upm.es/sitc.

This server contains a dataset about [Beatles songs](http://www.snee.com/bobdc.blog/2017/11/sparql-queries-of-beatles-reco.html), which we will query with SPARQL.

We will provide you some example code to get you started, the *question* you will have to answer using SPARQL, a template for the answer.

After every query, you will find some python code to test the results of the query.
**Make sure you've run the tests before moving to the next exercise**.
If the test gives you an error, you've probably done something wrong.
You do not need to understand or modify the test code.

For convenience, the examples in the notebook are executable (using the `%%sparql` magic command), and they are accompanied by some code to test the results.
If the tests pass, you probably got the answer right.

**Run this line to enable the `%%sparql` magic command.**

In [ ]:
from helpers import sparql, solution, show_photos

The `%%sparql` magic command will allow us to use SPARQL inside normal jupyter cells.

For instance, the following code:

```python 
%%sparql http://dbpedia.org/sparql

<MY QUERY>
```    

Is the same as `run_query('<MY QUERY>', endpoint='http://dbpedia.org/sparql')` plus some additional steps, such as saving the results in a nice table format so that they can be used later and storing the results in a variable (`solution()`), which we will use in our tests.

You do not need to worry about it, and **you can always use one of the suggested online editors if you wish**.

You can also use any other method to write your queries.
Just make sure to copy the working query back into the notebook so you can test it.

You may find online query editors particularly useful.
In addition to running queries from your browser, they provide useful features such as syntax highlighting and autocompletion.
Some examples are:

* DBpedia's virtuoso query editor https://dbpedia.org/sparql
* A javascript based client hosted at GSI: http://yasgui.cluster.gsi.dit.upm.es/

[^1]: http://www.snee.com/bobdc.blog/2017/11/sparql-queries-of-beatles-reco.html

## Exercises

The following exercises cover the basics of SPARQL with simple use cases.

#### First select - Exploring the dataset



Let's start with a simple query to explore the dataset using SPARQL.
We will get a list of the types of entities in the dataset.

SPARQL syntax is similar to SQL, mixed with turtle.
A SPARQL query has two main parts: the `SELECT` block, which specifies what variables we want to get; and the `WHERE` block which, loosely speaking, defines how the variables will be obtained from the graph.

In order to construct the `WHERE` block, we have to know the data we want to extract would be represented in Turtle.

In particular, to write an entity and its type, we would write this triple:

```turtle
<my_entity> a <type> .
```

For example:

```turtle
example:Timmy a example:Boy
```

In SPARQL, the parts that we wish to extract are replaced with a variable (e.g. `?name`, `?type`).
Hence, we would have something like this:

```turtle
?entity a ?type
```

The name of the variable has no effect on the query, but you should use a sensible name.
In these notebooks, try to use the names provided in the templates, because they might be used in the tests.

There are additional parts in the query.
For now, we will only cover the `LIMIT` statement, which limits the number of results we will get.
Using `LIMIT` is usually a good idea, especially when trying new queries, because the dataset may be too big. 

Using all these concepts, we will run our first query, to get the list of entities and their type:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

SELECT ?entity ?type
WHERE {
    ?entity a ?type
}
LIMIT 10

You can check that the results you got match our expectations:

In [ ]:
assert len(solution()['tuples']) == 10  # Make sure we got 10 results 
assert len(solution()['columns']) >= 1  # In 2 columns (?entity and ?type)

Now, use the same concepts to write a query that gets the **list of entities and their properties**.

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

SELECT DISTINCT ?entity ?prop
WHERE {
# YOUR ANSWER HERE
}
LIMIT 100

In [ ]:
s = solution()
assert len(s['tuples']) >= 100  # There are at least 100 results
assert 'entity' in s['columns']  # A column named entity exists
assert 'http://learningsparql.com/ns/musician/RaymondBrown' in s['columns']['entity'] # RaymondBrown is an entity

### Getting a list of DISTINCT types

To get a better grip of the dataset, we will get a list of types.

We may try to do so with a simple query: 

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

SELECT ?type
WHERE {
    ?entity a ?type
}
LIMIT 10

However, this list has many duplicates.
In fact, we only get one type (`Musician`).

To remove duplicates, we will need the `DISTINCT` statement, which only shows unique (distinct) rows:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

SELECT DISTINCT ?type
WHERE {
    ?entity a ?type
}
LIMIT 100

We should see only three types now (`Musician`, `Song`, and `Instrument`).

In [ ]:
assert 'type' in solution()['columns']
assert len(solution()['tuples']) == 3
assert 'http://learningsparql.com/ns/schema/Musician' in solution()['columns']['type']
assert 'http://learningsparql.com/ns/schema/Song' in solution()['columns']['type']
assert 'http://learningsparql.com/ns/schema/Instrument' in solution()['columns']['type']

Now, **build a query to get the list of unique properties**:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

SELECT DISTINCT ?property
WHERE {
# YOUR ANSWER HERE
}

In [ ]:
assert len(solution()['tuples']) == 182
assert 'http://learningsparql.com/ns/instrument/bass' in solution()['columns']['property']

### Geting all properties for songs

The `WHERE` statement can contain more than one line.

For example, we can restrict the list of properties from the previous exercise, to only get properties of musicians:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?prop
WHERE {
    ?song a s:Musician .
    ?song ?prop ?value .
}
LIMIT 20

There should be two results:

In [ ]:
assert len(solution()['tuples']) == 2 # There are exactly two results

Notice the use of prefixes, just like in turtle.
Also, these two options are equivalent:

```turtle
?song a s:Musician ;
     ?prop ?value .

# And

?song a s:Musician ;
?song ?prop ?value .
```

The first one is just shorter to write.

Alternatively, in this example we can also replace the properties we are not using with square brackets `[]`:

```turtle
[] a s:Musician ;
   ?prop [] .
```

Now, use the same concepts to get a list of **songs and properties**, without duplicates:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# YOUR ANSWER HERE
WHERE {
# YOUR ANSWER HERE
}
LIMIT 20

In [ ]:
s = solution()
assert len(set(s['tuples'])) == len(s['tuples'])  # There are no duplicates
assert len(s['tuples']) >= 20

### Getting a list of song names

In the previous exercise, we saw the properties for Songs.
One of them is `rdfs:label`, which gives a human readable name for the entity.

Using `rdfs:label`, get a list of song names:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name
WHERE {
# YOUR ANSWER HERE
}
LIMIT 20

In [ ]:
s = solution()
assert 'Besame Mucho' in s['columns']['name']

### Getting an ordered list of songs (ORDER BY)

The `ORDER BY` statement allows us to determine the way results will be sorted.
This makes it easier to find errors, or missing data.

The syntax is the following:

```sparql

SELECT *
WHERE { ... }
ORDER BY <variable> <variable> ... DESC(<variable>) ASC(<variable>)
... other statements like LIMIT ...
```

The results can be sorted in ascending or descending order, and using several variables.

Use `ORDER BY` to get a list of songs in **descending order**:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name
WHERE {
# YOUR ANSWER HERE
}
# YOUR ANSWER HERE
LIMIT 50

In [ ]:
s = solution()
assert len(s['tuples']) >= 20
assert s['columns']['name'][0][0] > s['columns']['name'][-1]

### Get a list of musicians who collaborated in at least one song (Traversing the graph)

From our inspection of the properties in previous exercises, we know that each song has a list of properties that link to musicians, and each musician has a name. For example:


```turtle
song:HeyJude a schema:Song ;
         instrument:guitar musician:RingoStarr .

musician:RingoStarr a schema:Musician ;
                    rdfs:label "Ringo Starr" .
```

Using this structure, and the SPARQL statements you already know, to get the **names** of all musicians that collaborated in at least one song.


In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?musician
WHERE {
    ?song a s:Song .
# YOUR ANSWER HERE
    ]
}
ORDER BY ?name

In [ ]:
s = solution()
assert 'musician' in s['columns']
assert 'Paul McCartney' in s['columns']['musician']
assert 'Peter Coe' in s['columns']['musician']
assert len(solution()['tuples']) >= 200

### In how many songs did Ringo collaborate? (COUNT)


Results can be aggregated using different functions.
One of the simplest functions is `COUNT`.
The syntax for COUNT is:
    
```sparql
SELECT COUNT(?variable) as ?count_name
```

Use `COUNT` and `GROUP BY` to get a 

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX m: <http://learningsparql.com/ns/musician/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# YOUR ANSWER HERE
WHERE {
    ?song a s:Song .
    ?song ?instrument m:RingoStarr .
}

In [ ]:
assert solution()['columns']['number'][0] == '412'

### Getting the frequency of each instrument (GROUP BY)

Results can be grouped by one or more of the variables.

Grouping is achieved with the `GROUP BY` statement. 
The syntax for `GROUP BY` is:

    
```sparql
SELECT GROUP BY ?variable1 ?variable2 ...
```

Once results are grouped, they can be aggregated using any aggregation function, such as `COUNT`.

Using `GROUP BY` and `COUNT`, get the count of songs that use each instrument:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX m: <http://learningsparql.com/ns/musician/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?instrument (COUNT(?song) as ?number)
WHERE {
    ?song a s:Song .
    ?song ?instrument m:RingoStarr .
}
# YOUR ANSWER HERE
ORDER BY DESC(?number)

In [ ]:
s = solution()
assert len(s['tuples']) == 37
assert s['columns']['number'][-1] == '1'
assert s['columns']['number'][0] == '233'

### How many different instruments are there in every song?

We can use other keywords inside our aggregation.
For example, we could use `DISTINCT` to remove duplicates before aggregating.

Here is an example, which shows the number of songs each musician collaborated in.
It has to use `DISTINCT` because some artists play multiple instruments in a song.

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?artist (COUNT(DISTINCT ?song) as ?number)
WHERE {
    ?artist a s:Musician .
    ?song ?instrument ?artist .
}
GROUP BY ?artist
ORDER BY DESC(?number)

Now, use the same principle to get the count of **different** instruments in each song.
Some songs have several musicians playing the same instrument, but we only care about *different* instruments in each song.

Use `?number` for the count.

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

# YOUR ANSWER HERE
WHERE {
    [] a           s:Song ;
       rdfs:label  ?song ;
       ?instrument ?musician .
}
# YOUR ANSWER HERE
ORDER BY DESC(?number)

In [ ]:
s = solution()
assert s['columns']['number'][0] == '27'

### Who is the vocalist in every song? (using OPTIONAL)

In this exercise, we will get a list of songs and their vocalists.

We coul start with this query:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX i: <http://learningsparql.com/ns/instrument/>
PREFIX m: <http://learningsparql.com/ns/musician/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?song ?vocalist
WHERE {
    ?song a s:Song .
    ?song i:vocals ?vocalist
}
LIMIT 100

However, there are some songs that do not have a vocalist (at least, in the dataset).
Those songs will not appear in the list above, because we they do not match part of the `WHERE` clause.

In these cases, we can specify optional values in a query using the `OPTIONAL` keyword.
When a set of clauses are inside an OPTIONAL group, the SPARQL endpoint will try to use them in the query.
If there are no results for that part of the query, the variables it specifies will not be bound (i.e. they will be empty).

To exemplify this, we can use a property that **does not exist in the dataset**:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX i: <http://learningsparql.com/ns/instrument/>
PREFIX m: <http://learningsparql.com/ns/musician/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?song ?musician
WHERE {
    ?song a s:Song .
    OPTIONAL {
        ?song i:a_made_up_instrument ?musician
    }
}
LIMIT 100

Although the property does not exist, the query will still return all the songs.
In the column for our instrument, it returns an empty value.

Now, use the same concept, to get a list of the **names** of the vocalists (if any) in each song.

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX i: <http://learningsparql.com/ns/instrument/>
PREFIX m: <http://learningsparql.com/ns/musician/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?song ?vocalist
WHERE {
    ?s a s:Song .
    ?s rdfs:label ?song .
# YOUR ANSWER HERE
}
LIMIT 100

In [ ]:
s = solution()
assert 'Paul McCartney' in s['columns']['vocalist']
assert 'Paul McCartney' in s['columns']['vocalist']
assert ('Besame Mucho', 'Paul McCartney') in s['tuples']
assert '' in s['columns']['vocalist']  # Some songs do not have a vocalist

### What songs do not have a vocalist? (Bound)

Now we only want to list those songs that **do not** have a vocalist.

To do so, we can copy the query from the previous exercise, and filter the results with the `BOUND` function.

`BOUND` will return `true` if the variable has a value, and `false` otherwise.

This is very useful for two purposes.
Firstly, it allows us to look for patterns that **do not occur** in the graph, such as missing properties.
For instance, we could search for the authors with missing birth information so we can add it.
Secondly, we can use bound in filters to get conditional filters.

Add a filter below to only get songs without a vocalist:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX i: <http://learningsparql.com/ns/instrument/>
PREFIX m: <http://learningsparql.com/ns/musician/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?song
WHERE {
    ?s a s:Song .
    ?s rdfs:label ?song .
    OPTIONAL {
    ?s i:vocals ?vocalist
     }
# YOUR ANSWER HERE
}
LIMIT 100

In [ ]:
s = solution()
assert len(s['tuples']) == 23

### Who played guitar OR bass in the most songs? (Advanced FILTER with GROUP)

In this exercise, we want a table with the name of musicians that played either the guitar (`i:guitar`) or the bass (`i:bass`), the instrument they played, and the times they played it.

If a musician played both instruments, it should appear twice.

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX i: <http://learningsparql.com/ns/instrument/>
PREFIX m: <http://learningsparql.com/ns/musician/>

SELECT ?musician ?instrument (COUNT(DISTINCT ?song) AS ?number)
WHERE {
  ?song ?ins ?player .
  ?ins rdfs:label ?instrument .
  ?player rdfs:label ?musician .
# YOUR ANSWER HERE
}
# YOUR ANSWER HERE

ORDER BY DESC(?instrument) DESC(?number)

In [ ]:
s = solution()
assert ('George Harrison', 'guitar', '27') in s['tuples']
assert ('Stuart Sutcliffe', 'bass', '3') in s['tuples']

### Who played the most instruments? (Advanced FILTER II)

Now, count how many instruments each musician have played in a song.

**Do not count lead (`i:vocals`) or backing vocals (`i:backingvocals`) as instruments**.

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX i: <http://learningsparql.com/ns/instrument/>
PREFIX m: <http://learningsparql.com/ns/musician/>

# YOUR ANSWER HERE
WHERE {
  ?song ?ins ?player .
  ?ins rdfs:label ?instrument .
  ?player rdfs:label ?musician .
# YOUR ANSWER HERE
}
GROUP BY ?musician
ORDER BY DESC(?instrument) DESC(?number)

In [ ]:
s = solution()
assert ('John Lennon', '52') in s['tuples']
assert ('Andy White', '2') in s['tuples']

### Which songs had Ringo in dums OR Lennon in lead vocals? (UNION)

We can merge the results of several queries, just like using `JOIN` in SQL.
The keyword in SPARQL is `UNION`, because we are merging graphs.

`UNION` is useful in many situations.
For instance, when there are equivalent properties, or when you want to use two search terms and FILTER would be too inefficient.

The syntax is as follows:

```sparql
SELECT ?title
WHERE  {
  { ?book dc10:title  ?title }
  UNION
  { ?book dc11:title  ?title }
  
  ... REST OF YOUR QUERY ...

}
```

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX i: <http://learningsparql.com/ns/instrument/>
PREFIX m: <http://learningsparql.com/ns/musician/>

SELECT DISTINCT ?song
WHERE {
# YOUR ANSWER HERE
}

In [ ]:
assert len(solution()['tuples']) == 246

### In how many songs has each musician collaborated at least 10 times? (HAVING)

You can filter results after an aggregation, using the `HAVING` statement.
Its syntax is:
    

```sparql
SELECT ...
WHERE ...
GROUP BY ...
HAVING (<statement>)
```

e.g.

```sparql
HAVING (?count > 10)
```

Use this new statement to get the list of artists that played at least 10 times with the Beatlest, and the number of times they did:

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/

PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#> 

SELECT ?musician (COUNT(DISTINCT ?song) AS ?number) 
WHERE {
  ?song ?instrument [
      rdfs:label ?musician  
  ]
}
GROUP BY ?musician
# YOUR ANSWER HERE
ORDER BY DESC(?number)

In [ ]:
s = solution()
assert len(s['tuples']) == 7
assert s['columns']['musician'][0] == 'Paul McCartney'
assert s['columns']['musician'][-1] == 'Mal Evans'

## **Optional** exercises

These are additional exercises that can be solved with more advanced concepts.

If you are curious, you could also check the notebook on Advanced SPARQL concepts.

### What instruments could each musician play? (GROUP_CONCAT)


Another option to aggregate results is to concatenate them.
You can do so with:

```sparql
GROUP_CONCAT(?name; separator=",")
```

Using `GROUP_CONCAT`, get a list of the instruments that each musician could play.

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX i: <http://learningsparql.com/ns/instrument/>
PREFIX m: <http://learningsparql.com/ns/musician/>

# YOUR ANSWER HERE

### What types of vocals are there? (REGEX)

In one of the exercises, we excluded lead and backing vocals from the list of instruments.
However, are those the only types of vocals?

You can check if a string or URI matches a regular expression with `regex(?variable, "<regex>")`.

In [ ]:
%%sparql http://fuseki.cluster.gsi.dit.upm.es/sitc/
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX s: <http://learningsparql.com/ns/schema/>
PREFIX i: <http://learningsparql.com/ns/instrument/>
PREFIX m: <http://learningsparql.com/ns/musician/>

# YOUR ANSWER HERE

## References

* [SPARQL queries of Beatles recording sessions](http://www.snee.com/bobdc.blog/2017/11/sparql-queries-of-beatles-reco.html)
* [RDFLib documentation](https://rdflib.readthedocs.io/en/stable/).
* [Wikidata Query Service query examples](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

## Licence
The notebook is freely licensed under under the [Creative Commons Attribution Share-Alike license](https://creativecommons.org/licenses/by/2.0/).  

© 2018 Universidad Politécnica de Madrid.